In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
X = np.array([1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5])/10
y = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])/10
LSTM_layer_num = 4 # number of LSTM layers
layer_size = [10,10,10,10] # number of nodes in each layer

model = Sequential()
model.add(Dense(layer_size[0], input_dim = 1, activation = "softmax"))
for i in range(1,LSTM_layer_num) :
    model.add(Dense(layer_size[i], activation = "softmax"))
model.add(Dense(1, activation = "softmax"))
model.compile(loss = 'mse', optimizer = 'adam',
            metrics=['accuracy'])
model.summary()

model_params = {'epochs':30,
                'validation_split':0.2,
                'validation_data':None}
model.fit(X, y, epochs = model_params['epochs'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                20        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 361
Trainable params: 361
Non-trainable params: 0
_________________________________________________________________
Train on 15 samples
Epoch 1/30


In [ ]:
data = pd.read_csv("carol_data.csv")
data = data.drop("Unnamed: 0", axis = 1)
all_text = ""
for i in range(len(data['song_lyrics'])):
    data['song_titles'][i] = data['song_titles'][i].lower()
    data['song_lyrics'][i] = data['song_lyrics'][i].lower()
    all_text += data['song_lyrics'][i] + '\n'
    if i == 1:
        break

In [ ]:
# Mapping chars to ints :
chars = sorted(list(set(all_text)))
int_chars = dict((i, c) for i, c in enumerate(chars))
chars_int = dict((i, c) for c, i in enumerate(chars))

In [ ]:
n_chars = len(all_text)
n_vocab = len(chars)
print('Total Characters :' , n_chars) # number of all the characters in lyricsText.txt
print('Total Vocab :', n_vocab) # number of unique characters

In [ ]:
# process the dataset:
seq_len = 100
data_X = []
data_y = []

for i in range(0, n_chars - seq_len, 1):
    # Input Sequeance(will be used as samples)
    seq_in  = all_text[i:i+seq_len]
    # Output sequence (will be used as target)
    seq_out = all_text[i + seq_len]
    # Store samples in data_X
    data_X.append([chars_int[char] for char in seq_in])
    # Store targets in data_y
    data_y.append(chars_int[seq_out])
n_patterns = len(data_X)
print( 'Total Patterns :', n_patterns)

In [ ]:
# Reshape X to be suitable to go into LSTM RNN :
X = np.reshape(data_X , (n_patterns, seq_len, 1))
# Normalizing input data :
X = X/ float(n_vocab)
# One hot encode the output targets :
y = np_utils.to_categorical(data_y)

In [ ]:
X.shape

In [ ]:
checkpoint_name = 'Weights-LSTM-improvement-{epoch:03d}-{loss:.5f}-bigger.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='loss', verbose = 1, save_best_only = True, mode ='min')
callbacks_list = [checkpoint]

In [ ]:
LSTM_layer_num = 4 # number of LSTM layers
layer_size = [10,10,10,10] # number of nodes in each layer
model = Sequential()
model.add(LSTM(layer_size[0], input_shape =(X.shape[1], X.shape[2]), return_sequences = True))
for i in range(1,LSTM_layer_num) :
    model.add(LSTM(layer_size[i], return_sequences=True))
model.add(Flatten())
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.summary()

In [ ]:
model_params = {'epochs':30,
                'batch_size':128,
                'callbacks':callbacks_list,
                'verbose':1,
                'validation_split':0.2,
                'validation_data':None,
                'shuffle': True,
                'initial_epoch':0,
                'steps_per_epoch':None,
                'validation_steps':None}
model.fit(X,
          y,
          epochs = model_params['epochs'],
           batch_size = model_params['batch_size'],
           callbacks= model_params['callbacks'],
           verbose = model_params['verbose'],
           validation_split = model_params['validation_split'],
           validation_data = model_params['validation_data'],
           shuffle = model_params['shuffle'],
           initial_epoch = model_params['initial_epoch'],
           steps_per_epoch = model_params['steps_per_epoch'],
           validation_steps = model_params['validation_steps'])